In [4]:
import sys
sys.path.append('../')
import ta
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#machine learning/statistical dependencies
import statsmodels.api as sm
from kneed import KneeLocator
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, silhouette_score
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation
from sklearn.manifold import TSNE
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.arima.model import ARIMA 
# from pykalman import KalmanFilter
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Dropout
import scipy.cluster.hierarchy as shc
import plotly.express as px
import gc
from collections import defaultdict
import seaborn as sns
import lightgbm as lgb

2024-04-04 04:53:20.842867: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-04 04:53:22.236631: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
from pair_trading_foundations.utils import GetSP500Data
import warnings
# The yfinance module has some deprecation warning
warnings.filterwarnings('ignore')

In [107]:
import requests
from datetime import datetime
earliest_date ='2020-04-01'# '2008-12-01'
todays_date = datetime.today().strftime('%Y-%m-%d')
session = requests.Session()
session.verify = False
print(f'Extracting date range: [ {earliest_date} ]-[ {todays_date} ]')
all_data = GetSP500Data(earliest_date,todays_date).get_all_sp_tickers().get_consolidated_data();

Extracting date range: [ 2020-04-01 ]-[ 2024-04-04 ]


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [108]:
final_data = all_data.copy()
final_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,GICS Sector,GICS Sub-Industry
0,2020-04-01,110.367889,113.578598,109.540131,111.321068,90.223351,4518010.0,MMM,Industrials,Industrial Conglomerates
1,2020-04-02,110.668900,115.560204,110.451508,115.309364,93.455788,4772399.0,MMM,Industrials,Industrial Conglomerates
2,2020-04-03,113.720734,114.916389,111.120399,111.864548,90.663818,4291607.0,MMM,Industrials,Industrial Conglomerates
3,2020-04-06,115.267555,118.102005,114.464882,117.642143,95.346458,6745081.0,MMM,Industrials,Industrial Conglomerates
4,2020-04-07,123.327759,125.953178,120.735786,120.903008,97.989326,6573575.0,MMM,Industrials,Industrial Conglomerates


In [109]:
# Examine Null Values in final_data. Examine Date, Close and daily_returns.
final_data.isna().sum()

Date                 0
Open                 0
High                 0
Low                  0
Close                0
Adj Close            0
Volume               0
Ticker               0
GICS Sector          0
GICS Sub-Industry    0
dtype: int64

In [110]:
df = final_data.pivot(index='Date', columns='Ticker', values='Close') 
df.head()

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2020-04-01,68.919998,10.69,60.227501,73.419998,NaN,76.570000,26.020000,154.679993,301.209991,84.089996,...,156.190002,16.120001,54.000000,58.080002,37.529999,59.770000,67.650002,87.019417,176.460007,111.919998
2020-04-02,72.290001,10.06,61.232498,75.129997,NaN,79.440002,26.320000,156.270004,303.959991,87.699997,...,158.410004,15.840000,52.669998,60.419998,40.400002,63.400002,66.050003,90.009712,174.240005,116.029999
2020-04-03,70.419998,9.39,60.352501,73.370003,NaN,79.449997,24.610001,152.149994,293.609985,86.559998,...,157.089996,15.340000,48.500000,57.860001,39.209999,60.500000,63.299999,89.330093,178.080002,114.610001
2020-04-06,74.360001,9.50,65.617500,75.730003,NaN,82.730003,27.030001,166.050003,319.130005,95.800003,...,175.360001,17.100000,56.730000,61.150002,40.470001,65.190002,70.500000,95.087379,191.850006,126.790001
2020-04-07,74.029999,10.22,64.857498,75.389999,NaN,81.930000,27.879999,164.119995,308.929993,96.250000,...,178.690002,17.290001,60.849998,59.230000,41.240002,66.190002,71.349998,98.941750,192.360001,126.110001


In [111]:
exp_data = df.copy()

# Null Values are Examined for Each Individual Stock on a Basis of Descriptive Statistics.

print('Null Values =',exp_data.isnull().values.any())

Null Values = True


In [112]:
 print('Data Shape Before Cleaning =', exp_data.shape)

missing_percentage = exp_data.isnull().mean().sort_values(ascending=False)
dropped_list = sorted(list(missing_percentage[missing_percentage > 0].index))
print(f"The following is a list of all stocks with ticker-level NA values: \n{dropped_list}")
exp_data.drop(labels=dropped_list, axis=1, inplace=True)

print('Data Shape After Cleaning =', exp_data.shape)

Data Shape Before Cleaning = (1008, 501)
The following is a list of all stocks with ticker-level NA values: 
['ABNB', 'CEG', 'GEHC', 'GEV', 'KVUE', 'SOLV', 'VLTO']
Data Shape After Cleaning = (1008, 494)


In [113]:
exp_data = exp_data.fillna(exp_data.median()) ### This step to ensure we did not miss any odd days, and if we did, it is standard industry practice to take the median

exp_data['year']=exp_data.index.year

exp_data

Ticker,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,year
Date,,,,,,,,,,,,,,,,,,,,,
2020-04-01,68.919998,10.69,60.227501,73.419998,76.570000,26.020000,154.679993,301.209991,84.089996,33.869999,...,16.120001,54.000000,58.080002,37.529999,59.770000,67.650002,87.019417,176.460007,111.919998,2020
2020-04-02,72.290001,10.06,61.232498,75.129997,79.440002,26.320000,156.270004,303.959991,87.699997,34.330002,...,15.840000,52.669998,60.419998,40.400002,63.400002,66.050003,90.009712,174.240005,116.029999,2020
2020-04-03,70.419998,9.39,60.352501,73.370003,79.449997,24.610001,152.149994,293.609985,86.559998,34.650002,...,15.340000,48.500000,57.860001,39.209999,60.500000,63.299999,89.330093,178.080002,114.610001,2020
2020-04-06,74.360001,9.50,65.617500,75.730003,82.730003,27.030001,166.050003,319.130005,95.800003,35.779999,...,17.100000,56.730000,61.150002,40.470001,65.190002,70.500000,95.087379,191.850006,126.790001,2020
2020-04-07,74.029999,10.22,64.857498,75.389999,81.930000,27.879999,164.119995,308.929993,96.250000,35.980000,...,17.290001,60.849998,59.230000,41.240002,66.190002,71.349998,98.941750,192.360001,126.110001,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-27,147.369995,15.30,173.309998,180.350006,113.480003,91.480003,340.940002,504.399994,193.330002,62.990002,...,35.930000,101.790001,53.380001,114.970001,129.380005,137.320007,132.610001,299.160004,168.509995,2024
2024-03-28,145.509995,15.35,171.479996,182.100006,113.660004,92.440002,346.609985,504.600006,197.789993,62.810001,...,35.910000,102.230003,53.750000,116.239998,129.240005,138.649994,131.979996,301.440002,169.210007,2024
2024-04-01,145.559998,15.41,170.029999,180.759995,112.089996,92.309998,339.170013,502.089996,196.619995,62.349998,...,35.560001,106.559998,53.279999,116.989998,128.639999,140.470001,129.750000,301.000000,167.020004,2024


In [128]:
# prepare clustering data by splitting train and test to match trading strategy later

cluster_train_size = int(len(exp_data) * 0.7)

# Split the data into training, validation and test sets based on these lengths:
cluster_train = exp_data[0:cluster_train_size]
print(f"The length of the training set is: {len(cluster_train)}.")

The length of the training set is: 705.


In [129]:
max(cluster_train.index)

Timestamp('2023-01-18 00:00:00')

In [130]:
cluster_train

Ticker,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,year
Date,,,,,,,,,,,,,,,,,,,,,
2020-04-01,68.919998,10.690000,60.227501,73.419998,76.570000,26.020000,154.679993,301.209991,84.089996,33.869999,...,16.120001,54.000000,58.080002,37.529999,59.770000,67.650002,87.019417,176.460007,111.919998,2020
2020-04-02,72.290001,10.060000,61.232498,75.129997,79.440002,26.320000,156.270004,303.959991,87.699997,34.330002,...,15.840000,52.669998,60.419998,40.400002,63.400002,66.050003,90.009712,174.240005,116.029999,2020
2020-04-03,70.419998,9.390000,60.352501,73.370003,79.449997,24.610001,152.149994,293.609985,86.559998,34.650002,...,15.340000,48.500000,57.860001,39.209999,60.500000,63.299999,89.330093,178.080002,114.610001,2020
2020-04-06,74.360001,9.500000,65.617500,75.730003,82.730003,27.030001,166.050003,319.130005,95.800003,35.779999,...,17.100000,56.730000,61.150002,40.470001,65.190002,70.500000,95.087379,191.850006,126.790001,2020
2020-04-07,74.029999,10.220000,64.857498,75.389999,81.930000,27.879999,164.119995,308.929993,96.250000,35.980000,...,17.290001,60.849998,59.230000,41.240002,66.190002,71.349998,98.941750,192.360001,126.110001,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-11,158.169998,15.340000,133.490005,157.169998,113.150002,62.849998,281.079987,342.929993,169.789993,86.680000,...,31.940001,96.239998,72.779999,111.370003,114.860001,129.690002,124.320000,299.790009,157.899994,2023
2023-01-12,156.490005,16.830000,133.410004,152.220001,111.360001,63.070000,283.839996,344.540009,168.669998,87.389999,...,32.130001,98.610001,71.949997,113.220001,116.190002,128.889999,123.199997,293.000000,159.910004,2023
2023-01-13,156.919998,17.020000,134.759995,153.600006,113.510002,63.759998,282.140015,344.380005,168.979996,88.400002,...,31.820000,100.250000,71.169998,113.150002,116.930000,130.220001,123.379997,293.700012,160.919998,2023


In [131]:
# calculate average returns by year by stock
returns_year = cluster_train.groupby('year').mean()

# calculate return percent change, then aggregated returns and volatility by stock
returns = returns_year.pct_change().mean()

returns = pd.DataFrame(returns)
returns.columns = ['returns']
returns['volatility'] = returns_year.pct_change().dropna().std()

In [132]:
returns

,returns,volatility
Ticker,,
A,0.190299,0.292082
AAL,0.110039,0.428350
AAPL,0.106681,0.265688
ABBV,0.199651,0.126023
ABT,0.044972,0.150764
...,...,...
XYL,0.161021,0.342719
YUM,0.121003,0.148482
ZBH,0.002912,0.195007


In [133]:
ret_data = returns.copy()
ret_data.describe()

,returns,volatility
count,494.000000,494.000000
mean,0.129000,0.265657
std,0.146352,0.177308
min,-0.200199,0.004887
25%,0.042667,0.137076
50%,0.106825,0.237052
75%,0.185040,0.348931
max,0.893810,1.401613


In [134]:
# Standardize The Clustering Dataset Along a t-Distributed Curve
scale = StandardScaler().fit(ret_data)

# Transform the Original Clustering Dataset to These Standardized Values
scaled_data = pd.DataFrame(scale.fit_transform(ret_data),columns = ret_data.columns, index = ret_data.index) 
X = scaled_data
X.describe()

,returns,volatility
count,4.940000e+02,4.940000e+02
mean,-2.157518e-17,-3.541926e-16
std,1.001014e+00,1.001014e+00
min,-2.251641e+00,-1.472208e+00
25%,-5.904920e-01,-7.259192e-01
50%,-1.516693e-01,-1.614945e-01
75%,3.833028e-01,4.701346e-01
max,5.231124e+00,6.413178e+00


In [135]:
n_clusters = 15
random_state = 15

# Algorithm:
# 1. Randomly pick k centroids from the examples ploted above as initial cluster centers (𝜇𝑗,𝑗∈1,...,𝑘)
# 2. Assign each example to the nearest centroid,  𝜇𝑗
# 3. Move the centroids to the center of the examples that were assigned to it
# 4. Repeat (2-3) until the cluster assignments do not change or a maximum number of iterations is reached.

def fit_kmeans(X, n_clusters, random_state):

    kmeans = KMeans(n_clusters=n_clusters,
                    init='k-means++',
                    n_init=15,
                    max_iter=300,
                    tol=1e-04)
    model = kmeans.fit(X)
    return model

model = fit_kmeans(X, n_clusters, random_state)
print(model)

## get cluster prediction and add to DataFrame
kX = X.copy()
kX['cluster'] = model.predict(X)
kX

KMeans(n_clusters=15, n_init=15)


,returns,volatility,cluster
Ticker,,,
A,0.419277,0.149183,1
AAL,-0.129689,0.918503,12
AAPL,-0.152652,0.000173,1
ABBV,0.483242,-0.788324,4
ABT,-0.574732,-0.648642,10
...,...,...,...
XYL,0.219019,0.435060,1
YUM,-0.054696,-0.661529,4
ZBH,-0.862410,-0.398862,10


In [140]:
counts = clustered_series.value_counts()

cluster_vis_list = list(counts[(counts<20) & (counts>1)].index)[::-1]

clust1 = pd.DataFrame(clustered_series,columns=['label'])
grouped_df = clust1.groupby('label')

In [142]:
final_data_train = final_data.copy()
final_data_train = final_data_train.loc[final_data_train.Date <= max(cluster_train.index)]

In [144]:
pivot_data = final_data_train.pivot(index='Ticker', columns='Date', values='Adj Close')
pivot_data.head()

Date,2020-04-01,2020-04-02,2020-04-03,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-13,2020-04-14,2020-04-15,...,2023-01-04,2023-01-05,2023-01-06,2023-01-09,2023-01-10,2023-01-11,2023-01-12,2023-01-13,2023-01-17,2023-01-18
Ticker,,,,,,,,,,,,,,,,,,,,,
A,67.127464,70.409821,68.588455,72.425972,72.104561,74.695374,76.292709,74.227852,76.779716,75.046021,...,150.331711,150.767853,146.366989,146.168762,153.860306,156.774353,155.109192,155.535400,154.028809,152.710541
AAL,10.690000,10.060000,9.390000,9.500000,10.220000,11.330000,12.510000,11.560000,11.950000,12.290000,...,13.590000,13.990000,14.180000,14.610000,15.190000,15.340000,16.830000,17.020000,17.080000,16.639999
AAPL,58.739861,59.720036,58.861782,63.996723,63.255489,64.874489,65.342644,66.625153,69.989929,69.351105,...,125.497505,124.166641,128.735245,129.261642,129.837662,132.578842,132.499390,133.840164,135.012100,134.287094
ABBV,61.654808,63.090801,61.612835,63.594650,63.309132,65.971153,66.970474,67.432327,69.997688,69.733482,...,155.845032,155.654617,158.567963,153.912308,151.989136,149.637482,146.302399,147.628769,146.888687,143.399796
ABNB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,88.720001,87.709999,88.519997,89.239998,88.540001,94.440002,98.489998,100.370003,101.269997,101.110001


In [145]:
# cluster evaluation and model selection
cluster_size_limit = 9999
counts = clustered_series.value_counts()
ticker_count = counts[(counts>1) & (counts<=cluster_size_limit)]
print ("Number of clusters: %d" % len(ticker_count))
print ("Number of Pairs: %d" % (ticker_count*(ticker_count-1)).sum())

Number of clusters: 10
Number of Pairs: 37350


In [146]:
# Provide a functional definition for the cointegrated augmented dickey fuller test
def find_cadf_pairs(data):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = {}
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            ols = sm.OLS(S1,S2).fit()
            pred = ols.predict(S2)
            adf = ts.adfuller(S1-pred)
            score = adf[0]
            pvalue = adf[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.03:
                pairs[(keys[i], keys[j])] = adf
    return score_matrix, pvalue_matrix, pairs

In [147]:
# Extract Cointegrated Pairs. This takes approximately 30-40 minutes. Again, this would take much longer if clustering analysis was not applied.
adf_dict = {}

for i, clust in enumerate(ticker_count.index):
    tickers = clustered_series[clustered_series == clust].index
    score_matrix, pvalue_matrix, adf_pairs = find_cadf_pairs(cluster_train[tickers])
    adf_dict[clust] = {}
    adf_dict[clust]['score_matrix'] = score_matrix
    adf_dict[clust]['pvalue_matrix'] = pvalue_matrix
    adf_dict[clust]['pairs'] = adf_pairs
    
adf_pairs = []   
for cluster in adf_dict.keys():
    adf_pairs.extend(adf_dict[cluster]['pairs'])
    
print ("Number of pairs:", len(adf_pairs))
print ("In those pairs, we found %d unique tickers." % len(np.unique(adf_pairs)))
print(adf_pairs)

KeyboardInterrupt: 

In [ ]:
top500_pairs = list(map(list, adf_pairs))
top500_pairs = [lst for lst in top500_pairs if lst and not any(x is None for x in lst)]
top500_pairs

In [46]:
# Modification of the ADF test from earlier to apply to the top pairs in each cluster. Pulling out the p-value only.
def find_cadf_pairs_validate(data):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = {}
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            ols = sm.OLS(S1,S2).fit()
            pred = ols.predict(S2)
            adf = ts.adfuller(S1-pred)
            score = adf[0]
            pvalue = adf[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.05:
                pairs[(keys[i], keys[j])] = adf
    pvalue_result = min(pvalue_matrix[0])
    return pvalue_result

In [72]:
# loop the best cluster list to filter exp_data df
# loop the filtered df through find_cadf_pairs_validate function to retrieve the pvalue_result
# append the pvalue_result to a new dataframe to rank the best pair based on p-value

def cluster_ranking(data):
    n = len(data)
    cluster_ranked = pd.DataFrame({'ticker1':[],'ticker2':[],'pvalue':[]})
    for i in range(n):
        pair_validate_df=exp_data.filter(items=data[i])
        p_value = find_cadf_pairs_validate(pair_validate_df)
        new_row = {'ticker1':top500_pairs[i][0],'ticker2':top500_pairs[i][1],'pvalue': p_value}
        cluster_ranked = cluster_ranked.append(new_row,ignore_index=True)
        new_row = {}
    cluster_ranked = cluster_ranked.sort_values(by='pvalue')
    return cluster_ranked


In [73]:
# display the best pair from each cluster, sorted by the lowest p-value
top500_df = cluster_ranking(top500_pairs)

In [75]:
top500_df = top500_df[0:500]

In [91]:
top500_df = top500_df.reset_index()

In [70]:
top500_df.to_csv('top500pairs.csv')